<a href="https://colab.research.google.com/github/martinfernandoortiz/geoestadistica_unlp/blob/main/practica3_parte1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 3 - Parte I
## Análisis de los datos espaciales
A partir de los datos filtrados de la tabla del SMN, visualizar la dis-
tribución de estaciones y sus variables climatológicas principales, apli-
cando los métodos de interpolación espacial vistos en clase:

Para ello, primeramente vamos a manejar la tabla, con el fin de en-
tender mejor la información disponible. Abrir el archivo de datos y
verificar los siguiente:
- Abrir la tabla filtrada desde algún directorio especı́fico.
- Visualizar la cantidad de filas y columnas de la tabla. Mostrar en
una lista los nombres de las columnas.
- Visualizar sòlo las primeros 10 filas.
- Visualzar sòlo las últimas 10 filas.
- Visualzar el rango de filas: 10 - 20 y 40-70, en simultaneo (ayuda:
``` python
df.iloc[np.r[ 10 : 21, 40 : 71], :]
```
)
### Filtrado de datos:
Seleccionar todas las estaciones meteorológicas ubicadas a una altura mayor a 800 m. El resultado entregarlo como una lista con los nombres de las estaciones seleccionadas
De estas estaciones, identificar cuàles presentan los siguientes rasgos:

- 1 Mayor temperatura

- 2 Menor nubosidad total

- 3 Menor temperatura o menor humedad relativa

- Identificar en todas las estaciones, para cada mes, los estimadores estadı́sticos mas comunes, (media, std, cuartiles, mínimos y máximos)
- Por Estacion, hacer un promedio anual de la temperatura máxima y mínima
- Agregar una columna booleana (valores 0 o 1), que indiquen si la temperatura media de esta estación, en ese mes es  Superior o inferior a 15 ºC.
- Repetir lo anterior, pero esta vez indicando un valor inferior o su- perior de precipitación de 70 mm2. Visualzar sòlo las últimas 10 filas.


Para ello, primeramente vamos a manejar la tabla, con el fin de en-
tender mejor la información disponible. Abrir el archivo de datos y
verificar los siguiente:
- Abrir la tabla filtrada desde algún directorio especı́fico.


In [1]:
# @title
#Carga de Librerìas
################################################################
#
#   - Importacion de librerias
#   - Asignaciòn de alias
#
####################################################################

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import seaborn as sns
#import geopandas as gpd
# Preprocesado y modelado
# ==============================================================================
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')
#https://towardsdatascience.com/detecting-and-treating-outliers-in-python-part-3-dcb54abaf7b0

#Import Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
# @title
# Carga de datos

#path = "datasets/"
archivo = "SMN_estaciones_medias_mensuales_1989_2010.xls"
#df = pd.read_excel( 'SMN_estaciones_medias_mensuales_1989_2010.xls')
df = pd.read_excel('/content/drive/My Drive/Colab Notebooks/SMN_estaciones_medias_mensuales_1989_2010.xls')
df.replace("S/D", np.NaN, inplace = True) #reemplazar los "S/D"

#Finalmente vamos a utilizar ésta forma para cambiar el tipo de campo
rango= np.r_[8:16]
df[df.columns[rango]] = df[df.columns[rango]].astype(float)
df.dtypes
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 16 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Unnamed: 0.1                           840 non-null    int64  
 1   Unnamed: 0                             840 non-null    int64  
 2   Estacion                               840 non-null    object 
 3   mes                                    840 non-null    object 
 4   lat                                    840 non-null    float64
 5   long                                   840 non-null    float64
 6   Altura_(m)                             840 non-null    int64  
 7   Numero                                 840 non-null    int64  
 8   Temperatura_(C)                        838 non-null    float64
 9   Temperatura_maxima_(C)                 840 non-null    float64
 10  Temperatura_minima_(C)                 818 non-null    float64
 11  Humeda

- Visualizar la cantidad de filas y columnas de la tabla. Mostrar en
una lista los nombres de las columnas.



In [38]:
print(df.info())
print(df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 16 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Unnamed: 0.1                           840 non-null    int64  
 1   Unnamed: 0                             840 non-null    int64  
 2   Estacion                               840 non-null    object 
 3   mes                                    840 non-null    object 
 4   lat                                    840 non-null    float64
 5   long                                   840 non-null    float64
 6   Altura_(m)                             840 non-null    int64  
 7   Numero                                 840 non-null    int64  
 8   Temperatura_(C)                        838 non-null    float64
 9   Temperatura_maxima_(C)                 840 non-null    float64
 10  Temperatura_minima_(C)                 818 non-null    float64
 11  Humeda


- Visualizar sòlo las primeros 10 filas.

- Visualzar sòlo las últimas 10 filas.
- Visualzar el rango de filas: 10 - 20 y 40-70, en simultaneo (ayuda:
``` python
df.iloc[np.r[ 10 : 21, 40 : 71], :]
```
)

> Indented block




In [6]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Estacion,mes,lat,long,Altura_(m),Numero,Temperatura_(C),Temperatura_maxima_(C),Temperatura_minima_(C),Humedad_relativa_(%),Velocidad_del_Viento_(km/h),Nubosidad_total_(octavos),Precipitacion_(mm),Frecuencia_dias_Precipitacion_>0.1_mm
0,0,0,LA QUIACA,ene,-12.0,-63.333333,3459,87007,12.8,20.1,7.4,65.3,6.9,5.2,97.5,15.9
1,1,1,LA QUIACA,feb,-12.0,-63.333333,3459,87007,12.5,19.9,7,64.1,7.1,4.8,68.4,12.2
2,2,2,LA QUIACA,mar,-12.0,-63.333333,3459,87007,12.4,19.9,6.3,62.4,6.5,4.1,55.9,9.8
3,3,3,LA QUIACA,abril,-12.0,-63.333333,3459,87007,10.9,19.7,2.8,48.2,6.4,2.7,8.2,2.4
4,4,4,LA QUIACA,mayo,-12.0,-63.333333,3459,87007,6.9,17.3,-3,34.6,6,1.8,1,0.3


- Visualzar sòlo las últimas 10 filas.


In [7]:
df.tail()

,Unnamed: 0.1,Unnamed: 0,Estacion,mes,lat,long,Altura_(m),Numero,Temperatura_(C),Temperatura_maxima_(C),Temperatura_minima_(C),Humedad_relativa_(%),Velocidad_del_Viento_(km/h),Nubosidad_total_(octavos),Precipitacion_(mm),Frecuencia_dias_Precipitacion_>0.1_mm
835,835,847,BASE SAN MARTIN,ago,-60.5,-59.5,7,89066,-11.5,-6.9,-16.8,NaN,18.9,5.9,NaN,NaN
836,836,848,BASE SAN MARTIN,sep,-60.5,-59.5,7,89066,-8.9,-4.6,-13.9,NaN,17.5,6.3,NaN,NaN
837,837,849,BASE SAN MARTIN,oct,-60.5,-59.5,7,89066,-5.9,-2.1,-10.2,NaN,20.3,6.4,NaN,NaN
838,838,850,BASE SAN MARTIN,nov,-60.5,-59.5,7,89066,-2,1.2,-5.5,NaN,19.2,6.4,NaN,NaN
839,839,851,BASE SAN MARTIN,dic,-60.5,-59.5,7,89066,0.8,3.4,-1.8,NaN,18.3,6.2,NaN,NaN


- Visualzar el rango de filas: 10 - 20 y 40-70, en simultaneo (ayuda:

In [13]:
#rango=np.r_[ 10:21,40:71]
df.iloc[np.r_[ 10:21,40:71], :]


,Unnamed: 0.1,Unnamed: 0,Estacion,mes,lat,long,Altura_(m),Numero,Temperatura_(C),Temperatura_maxima_(C),Temperatura_minima_(C),Humedad_relativa_(%),Velocidad_del_Viento_(km/h),Nubosidad_total_(octavos),Precipitacion_(mm),Frecuencia_dias_Precipitacion_>0.1_mm
10,10,10,LA QUIACA,nov,-12.000000,-63.333333,3459,87007,12.8,21.7,5.2,52.3,8.8,3.6,27.3,7.4
11,11,11,LA QUIACA,dic,-12.000000,-63.333333,3459,87007,13.2,21.3,6.9,59.2,8.3,4.5,71.9,12.7
12,12,12,ORAN,ene,-16.333333,-60.842105,357,87016,NaN,32.3,21.5,NaN,8.4,NaN,216.6,14.3
13,13,13,ORAN,feb,-16.333333,-60.842105,357,87016,25.1,31.1,20.8,NaN,8.4,NaN,183.7,12.9
14,14,14,ORAN,mar,-16.333333,-60.842105,357,87016,23.9,29.4,20.5,83.6,6.9,5.7,184.2,15.3
15,15,15,ORAN,abril,-16.333333,-60.842105,357,87016,20.9,26.0,17.8,85.3,6.4,6,70.9,12
16,16,16,ORAN,mayo,-16.333333,-60.842105,357,87016,17.7,23.2,14.2,83.8,6.2,5.6,19.8,6.6
17,17,17,ORAN,jun,-16.333333,-60.842105,357,87016,15,21.2,11,82.3,6.8,5.1,6.3,3.9
18,18,18,ORAN,jul,-16.333333,-60.842105,357,87016,14.7,22.0,9.2,75.4,7.7,4,4.5,2.8
19,19,19,ORAN,ago,-16.333333,-60.842105,357,87016,17.2,25.6,11,67.3,8.2,3.5,5.4,2.2


### Filtrado de datos:
Seleccionar todas las estaciones meteorológicas ubicadas a una altura mayor a 800 m. El resultado entregarlo como una lista con los nombres de las estaciones seleccionadas
De estas estaciones, identificar cuàles presentan los siguientes rasgos:

- 1 Mayor temperatura

- 2 Menor nubosidad total

- 3 Menor temperatura o menor humedad relativa

- Identificar en todas las estaciones, para cada mes, los estimadores estadı́sticos mas comunes, (media, std, cuartiles, mínimos y máximos)
- Por Estacion, hacer un promedio anual de la temperatura máxima y mínima
- Agregar una columna booleana (valores 0 o 1), que indiquen si la temperatura media de esta estación, en ese mes es  Superior o inferior a 15 ºC.
- Repetir lo anterior, pero esta vez indicando un valor inferior o su- perior de precipitación de 70 mm2. Visualzar sòlo las últimas 10 filas.

- 1 Mayor temperatura

In [39]:
# Selecciono e imprimo las estaciones de mas de 800 metros
estaciones=df[df['Altura_(m)']>800].iloc[:,2].drop_duplicates()
print(estaciones)

0                 LA QUIACA
36                    JUJUY
48                    SALTA
372    MENDOZA OBSERVATORIO
444                MALARGUE
708               BARILOCHE
720              MAQUINCHAO
Name: Estacion, dtype: object


Saco la mayor temperatura, menor nubosidad total y menor temperatura o menor humedad relativa


In [40]:
# @title
df800= df[df['Altura_(m)']>800]
# Seleccionar las variables relevantes para el clustering
campos = df800[['Temperatura_(C)', 'Temperatura_maxima_(C)', 'Temperatura_minima_(C)',
                            'Humedad_relativa_(%)', 'Velocidad_del_Viento_(km/h)',
                            'Nubosidad_total_(octavos)', 'Precipitacion_(mm)',
                            'Frecuencia_dias_Precipitacion_>0.1_mm']]



In [41]:
# @title
# Encontrar la estación con el máximo valor en 'a'
estacion_max_a = df800.loc[df800['Temperatura_maxima_(C)'].idxmax(), 'Estacion']
indice_max_a = df800['Temperatura_maxima_(C)'].idxmax()
# Imprimir el resultado
print(f"Estación con el máximo valor de temperatura es {estacion_max_a} con  {indice_max_a} grados " )

Estación con el máximo valor de temperatura es JUJUY con  47 grados 


In [43]:
# @title
# Encontrar la estación con el máximo valor en 'a'
estacion_max_a = df800.loc[df800['Nubosidad_total_(octavos)'].idxmin(), 'Estacion']
indice_max_a = df800['Nubosidad_total_(octavos)'].idxmin()
# Imprimir el resultado
print(f"Estación con el mínimo de nubosidad es {estacion_max_a} con  {indice_max_a}  " )

Estación con el mínimo de nubosidad es LA QUIACA con  6  


In [47]:
# @title

minTemp = df800.loc[df800['Temperatura_minima_(C)'].idxmin(), 'Estacion']
minHumedad = df800.loc[df800['Humedad_relativa_(%)'].idxmin(), 'Estacion']
#print(minTemp + minHumedad)
temp = df800['Temperatura_minima_(C)'].idxmin()

hum = df800['Humedad_relativa_(%)'].idxmin()

print(f"La quiaca tiene la menor temperatura ({temp}º ) y menor humedad relativa ({hum} )    " )

LA QUIACALA QUIACA
La quiaca tiene la menor temperatura (6º ) y menor humedad relativa (6 )    


- Identificar en todas las estaciones, para cada mes, los estimadores estadı́sticos mas comunes, (media, std, cuartiles, mínimos y máximos)

In [56]:
stats_por_mes = df800.groupby(['Estacion','mes']).describe()
print(stats_por_mes)

                Unnamed: 0.1                                                \
                       count   mean std    min    25%    50%    75%    max   
Estacion  mes                                                                
BARILOCHE abril          1.0  711.0 NaN  711.0  711.0  711.0  711.0  711.0   
          ago            1.0  715.0 NaN  715.0  715.0  715.0  715.0  715.0   
          dic            1.0  719.0 NaN  719.0  719.0  719.0  719.0  719.0   
          ene            1.0  708.0 NaN  708.0  708.0  708.0  708.0  708.0   
          feb            1.0  709.0 NaN  709.0  709.0  709.0  709.0  709.0   
...                      ...    ...  ..    ...    ...    ...    ...    ...   
SALTA     mar            1.0   50.0 NaN   50.0   50.0   50.0   50.0   50.0   
          mayo           1.0   52.0 NaN   52.0   52.0   52.0   52.0   52.0   
          nov            1.0   58.0 NaN   58.0   58.0   58.0   58.0   58.0   
          oct            1.0   57.0 NaN   57.0   57.0   57.0   5

In [62]:

# Rellenar los valores nulos en 'Temperatura_(C)', 'Temperatura_minima_(C)' con la media
df['Temperatura_(C)'].fillna(df['Temperatura_(C)'].mean(), inplace=True)
df['Temperatura_minima_(C)'].fillna(df['Temperatura_minima_(C)'].mean(), inplace=True)

# Rellenar los valores nulos en otras columnas según tus necesidades

# Calcula estadísticas descriptivas para cada mes
estadisticas_por_mes = df.groupby('mes').describe()

print("Estadísticas descriptivas para cada mes:")
print(estadisticas_por_mes)

# Calcula promedio anual de la temperatura máxima y mínima por estación
promedio_anual = df.groupby(['Estacion']).agg({
    'Temperatura_maxima_(C)': 'mean',
    'Temperatura_minima_(C)': 'mean'
}).reset_index()

print("\nPromedio anual de la temperatura máxima y mínima por estación:")
print(promedio_anual)

# Agrega una columna booleana que indica si la temperatura media en ese mes es superior a 15 ºC
df['Temperatura_media_superior_15'] = df['Temperatura_(C)'].gt(15)

# Repite para precipitación mayor a 70 mm2
df['Precipitacion_superior_70'] = df['Precipitacion_(mm)'].gt(70)

# Visualiza solo las últimas 10 filas
ultimas_10_filas = df.tail(10)
print("\nVisualizar solo las últimas 10 filas:")
print(ultimas_10_filas)

Estadísticas descriptivas para cada mes:
      Unnamed: 0.1                                                             \
             count        mean         std   min     25%    50%    75%    max   
mes                                                                             
abril         70.0  416.857143  244.213039   3.0  210.75  417.0  624.0  831.0   
ago           70.0  421.085714  244.206894   7.0  215.75  421.0  628.0  835.0   
dic           70.0  425.314286  244.204330  11.0  220.75  425.0  632.0  839.0   
ene           70.0  413.685714  244.219997   0.0  207.00  414.0  621.0  828.0   
feb           70.0  414.742857  244.217454   1.0  208.25  415.0  622.0  829.0   
jul           70.0  420.028571  244.208094   6.0  214.50  420.0  627.0  834.0   
jun           70.0  418.971429  244.209519   5.0  213.25  419.0  626.0  833.0   
mar           70.0  415.800000  244.215134   2.0  209.50  416.0  623.0  830.0   
mayo          70.0  417.914286  244.211167   4.0  212.00  418.0  625